# Pas 1: Se transforma imaginea intr-o reprezentare secventiala (similar textelor) <img  style="vertical-align:middle" src="images/ViT/patchify2.png" width="600"/>

## 1.1 fiecare imagine este “sparta” in patch-uri



De ce e necesar?
- pentru a putea prelucra imagini mai mari prin prelucrarea paralela a patch-urilor
- pentru a avea "acoeprire" mai mare decat o simpla convolutie, dar cu pastrarea caracterului local al feature-urilor extrase 
- pentru a avea modele generale care pot fi folosite pe imagini de dimensiuni diferite (dar a caror patch-uri au aceleasi dimensiuni)

Specificatii:
- Input: <img  style="vertical-align:middle" src="images/ViT/bike.png" width="200"/>
    - imaginea RGB de dimensiune 3 X W x H
    - numarul de patch-uri (pe W, pe H): n_patches

- Output: <img  style="vertical-align:middle" src="images/ViT/bikePatchesWithFrames2.png" width="200"/>
    - vector in 2 dimensiuni cu patch-uri
        - n_patches x n_patches
        - fiecare patch are dimensiunea W / n_patches x H / n_patches

Cum?
- prin divizare 
    
    > in implementare liniile 23-30

## 1.2 matricea de patch-uri este liniarizata



De ce e necesar?
- pentru a avea o reprezentare secventiala (similara secventelor de cuvinte)

Specificatii:
- Input: <img  style="vertical-align:middle" src="images/ViT/bikePatchesWithFrames2.png" width="150"/>
    - vector in 2 dimensiuni cu patch-uri
        - no total de patchuri: n_patches x n_patches
        - Fiecare patch are dimensiunea Dp x Dp = W / n_patches x H / n_patches


- Output: <img  style="vertical-align:middle" src="images/ViT/bikePatchesWithFrames1Row2.png" width="400"/>
    - vector in 1 dimensiune cu patch-uri
        - no total de patchuri: n_patches x n_patches
        - fiecare patch are dimensiunea Dp x Dp = W / n_patches x H / n_patches

Cum?
- prin concatenarea patch-urilor intr-o lista de patch-uri 

    > in implementare linia 31


# Pas 2: se aplica o transformare liniara asupra fiecarui patch <img  style="vertical-align:middle" src="images/ViT/patchLin2.png" width="600"/>

## 2.1 fiecare patch este "aplatizat" (flattening)



De ce e necesar?
- pentru a avea o reprezentare cu o singura dimensiune (array, nu matrice)

Specificatii:
- Input: patch = vector in 3 dimensiuni (cub) cu 3 linii, DP coloane si Dp adancime care contine 3 x Dp x Dp elemente
- Output: vector in 2 dimensiuni cu 3 linii si (Dp * Dp) coloane 

Cum?
- prin concatenarea liniilor fiecarui patch

    > in implementare linia 31

## 2.2 fiecare patch aplatizat este mapat (proiectat) intr-o reprezentare cu un nr diferit de dimensiuni

De ce e necesar?
- modificarea (in special) reducerea dimensiunii
- pt standardizare, adica lucrul cu o reprezentare de lungime fixa, indiferent de dimensiunea patch-ului / imaginii
- transformarea liniara = aplicarea unui filtru, deci extragerea / evidentierea unor caracteristicii ale patch-ului/imaginii

Specificatii:
- Input: vector intr-o dimensiune cu (Dp x Dp) elemente 
- Output: vector intr-o dimensiune cu De elemente 

Cum?
- elemente algebrice:
    Daca presupunem:
    - $x$ un array cu $d1$ elemente si 
    - vrem sa-l mapam intr-un array $y$ cu $d2$ elemente, 

    atunci putem folosi o matrice $A$ de dimensiune $d1 \times d2$ si sa facem $ x \cdot A = y $ pentru ca:

    $ [x_1 \hspace{0.15cm} x_2 \hspace{0.15cm}\ldots \hspace{0.15cm} x_{d1}] \times 
    \begin{bmatrix}
    a_{1,1} & a_{1,2} &  \ldots & a_{1,d2} \\ 
    a_{2,1} & a_{2,2} &  \ldots & a_{2,d2} \\ 
    & & \ldots & \\ 
    a_{d1,1} & a_{d1,2} &  \ldots & a_{d1,d2}
    \end{bmatrix}
     = [ y_1 \hspace{0.15cm} y_2 \hspace{0.15cm} \ldots \hspace{0.15cm} y_{d2}]$

    Sau, mai general, putem folosi o matrice $A$ de dimensiune $d1 \times d2$ si un vector $b$ (bias) cu $d2$ valori a.i. sa facem $ x \cdot A + b= y $ pentru ca:

    $ [x_1 \hspace{0.15cm} x_2 \hspace{0.15cm}\ldots \hspace{0.15cm} x_{d1}] \times 
    \begin{bmatrix}
    a_{1,1} & a_{1,2} &  \ldots & a_{1,d2} \\ 
    a_{2,1} & a_{2,2} &  \ldots & a_{2,d2} \\ 
    & & \ldots & \\ 
    a_{d1,1} & a_{d1,2} &  \ldots & a_{d1,d2}
    \end{bmatrix} + [b_1 \hspace{0.15cm} b_2 \hspace{0.15cm} \ldots \hspace{0.15cm} b_{d2}] = [ y_1 \hspace{0.15cm} y_2 \hspace{0.15cm} \ldots \hspace{0.15cm} y_{d2}]$

- elemente de ML:
    - transformarea lui $x$ in $y$ prin matricea A si vectorul b se numeste "forward pass" in retelele neuronale, adica corespunde "trecerii" valorilor din vectorul $x$ peste un strat dintr-o retea neuronala cu un sg strat ascuns - o retea ce primeste exemple cu $d1$ atribute (deci $d1$ neuroni pe stratul de intrare) si $d2$ neuroni pe stratul ascuns, fiecare cu o functie de activare liniara
    - coeficientii stratului ascuns sunt elementele matricii $A$ si urmeaza sa fie descoperiti prin procesul de antrenare a transformer-ului
    - deci daca avem un patch "aplatizat" (cu o singura linie) cu $Dp \times Dp$ elemente si vrem sa-l mapam intr-un vector de dimensiune $De$, atunci avem nevoie de o matrice $A$ cu $Dp \times Dp$ linii si $ De$ coloane si un vector coloana $b$ cu $De$ elemente.

- Atentie: toate patch-urile trebuie transformate, nu doar unul singur

    > in implementare liniile 120-121 si linia 148: 
        - se foloseste un modul nn.Linear care primeste ca parametrii:
            - numarul de neuroni pe stratul de intrare (adica nr de pixeli dintr-un patch - $Dp \times Dp$) si
            - numarul de neuroni pe stratul ascuns ($De$ in cazul nostru)
        - si are rolul sa invete coeficientii matricii de transformare A (eventual si bias-ul b) - valoriile matricii A reprezinta coeficientii retelei neuronale (pt acel strat) care se invata in timpul antrenarii
        - in figura, fiecare patch e supus unui "linear embedding" sau "linear projection"; pt fiecare patch se obtine un token (vector) de dimensiune 8, deci pentru o imagine vom avea 49 de tokens de lungime 8, iar pentru tot datasetul vom avea (N, 49, 8) tokens <img  style="vertical-align:middle" src="images/ViT/ViT2.png" alt="softmaxLoss" width="600"/>
<!-- <img  style="vertical-align:middle" src="images/ViT/ViT.png" alt="softmaxLoss" width="600"/> -->


# Pas 3 - calcularea unor embeddinguri de pozitie pt fiecare patch  <img  style="vertical-align:middle" src="images/ViT/patchLinPos3.png" width="600"/>

De ce e necesar?
- este util pt localizarea patch-ului (relativ la imaginea originala) - similar cu a tine cont de relatiile dintre cuvinte intr-o secventa de text. <img  style="vertical-align:middle" src="images/ViT/patchOrder.png" width="200"/>
- Reminder: in transformers, nu se tine cont de ordinea token-urilor (cuvintelor, patch-urilor) dintr-o secventa, ci de relatia dintre ele (prin self-attention)


Specificatii:
- Input: 
    - pt fiecare patch, un vector intr-o dimensiune cu De elemente 
    - numarul total de patch-uri (tokens) - atentie, doar nr - si nr de dimensiuni ale token-ului (De = ? in cazul nostru)
- Output: 
    - vector intr-o dimensiune cu De elemente (imbogatite/imbunatatite)

Cum? 
- Folosind acelasi principiu ca la text (bazat pe sin si cos) - pentru fiecare token se adauga un vector de dimensiune $De$ care contine informatii despre pozitia token-ului in imagine:

$ pos_{i,j} = \left\{ 
    \begin{array}{ll}
    \sin{\frac{i}{1000^{\frac{j}{De}}}}, & \text{if } j \text{ is even} \\
    \cos{\frac{i}{1000^{\frac{j - 1}{De}}}}, & \text{if } j \text{ is odd}
    \end{array}
    \right. 
  $ unde $i$ este indexul token-ului (patch-ului) si $j$ este indexul elementului din vectorul de dimensiune $De$ asociat token-ului <img  style="vertical-align:middle" src="images/ViT/posEmbeddings.png" width="400"/>

- agregarea (prin adunare) a embeddingurilor de patch-uri cu embeddingurile de pozitie

    > in implementare, clasa MyVit, liniile 127-131 si linia 154 (adunarea embeddingurilor de patch-uri cu cele de pozitie) 

# Pas 4 - adaugarea unui token de clasificare ("[CLS] token") la inceputul secventei de token-uri   <img  style="vertical-align:middle" src="images/ViT/clsToken.png" width="600"/>

De ce e necesar?
- pentru a agrega informatii despre intreaga secventa de patch-uri
- este util modelului de clasificare pentru a lua decizii pe baza informatiilor agregate (e folosit ca input in head-ul de clasificare) si foarte utila in task-uri de transfer learning
- se comporta ca oricare alt token (in etapa de calculare a atentiei), dar este folosit pentru a clasifica imaginea
- pt eficienta: are o reprezentare unica si de lungime fixa pentru orice secventa de patch-uri (sau cuvinte/texte)
- pt flexibilitate: permite modelului sa se antreneze pe mai multe task-uri fara a modifica arhitectura modelului
- se initializeaza ca un token special cu un embedding unic (indiferent de continutul patch-urilor, respectiv al embedding-urilor asociate acestor patch-uri si nu are componenta de "positional embedding")

Specificatii:
- Input: numarul de elemente al unui embedding de patch (De = ? in cazul nostru)
- Output: pentru fiecare secventa de patch-uri, cate un token de clasificare cu un embedding cu De elemente

Cum?
- initializare aleatoare a vectorului cu De elemente

    > in implementare, in clasa MyViT, linia 124 si linia 151

# Pas 5 - blocul de transformer (Encoder-ul)  <img  style="vertical-align:middle" src="images/ViT/transformer.png" width="600"/> <img  style="vertical-align:middle" src="images/ViT/encoder.png" width="200"/>

De ce e necesar?
- pentru a invata relatii dintre patch-uri (similar cu invatarea relatiilor dintre cuvinte in NLP)

Specificati:
- input: Proceseaza ca input vectori (embedding-urile anterior calculate, fiecare cu cate De elemente)
- output: Returneaza ca output vectori (embedding-urile rezultate, fiecare cu cate De elemente)

Cum?
- similar cu blocul de transformer din NLP
- 3 layere principale: normalizare, self-attention, conexiuni reziduale

## 5.1 Normalizarea

De ce e necesara?
- pentru a compara "mere cu mere si pere cu pere" la nivelul fiecarei secvente de patch-uri

Specificatii:
- Input: $n + 1$ vectori cu De elemente (n patch-uri plus un token de clasificare reprezentate prin De elemente)
- Output: $n + 1$ vectori cu De elemente (n patch-uri plus un token de clasificare)

Cum?
- normalizare statistica $x' = \frac{x- \mu}{\sigma}$, unde $\mu$ este media si $\sigma$ este deviatia standard a vectorului $x$ 
    
    > in implementare, linia 83 (in clasa MyViTBlock)

## 5.2 Multi-Head Self-Attention <img  style="vertical-align:middle" src="images/ViT/MSA.png" width="600"/>

De ce e necesar?
- deoarece vrem ca reprezentarea fiecarui patch (embedding-ul) sa fie actualizat pe baza similaritatii patch-ului cu celelalte patch-uri din imaginea curenta
- e responsabil cu "procesarea" localizarii spatiale a informatiei din imagine (urmand ca MLP-ul din head sa proceseze informatiile din mai multe canale ale imaginii)
    - element diferit de convolutii (cu filtre de dimensiune > 1) care proceseaza informatii de pe canale su locatii diferite ale imaginii deodata

Specificatii:
- Input: $n + 1$ vectori cu De elemente (n patch-uri plus un token de clasificare reprezentate prin De elemente) - se poate nota inputul cu x (o matrice cu n + 1 linii si De coloane)
- Output: $n + 1$ vectori cu De elemente (n patch-uri plus un token de clasificare)

Cum?
- pentru un mecanism de atentie (cu un singur head) am avea:
    - prin maparea fiecarui patch (reprezentat prin cele De elemente) in 3 vectori (query, key si value), calcularea produsului scalar intre query si key si normalizarea producsului (obtinandu-se indicii de atentie), aplicarea unui softmax (ponderi de atentie) si inmultirea cu value; in final agregarea rezultatelor. Altfel spus::
        - Calcularea a 3 vectori pt fiecare patch (aplatizat si proiectat linear in reprezentarea cu De elemente): 
            - Query, Key, Value. 
            - Cei 3 vectori au dimensiunea D (D poate fi diferit de De)
            - Cum? Prin folosirea a 3 matrici WQ, WK si WV de dimensiune De x D; Pt fiecare patch: <img  style="vertical-align:middle" src="images/ViT/QKV.png" width="200"/>
                - qi = xi * WQ
                - ki = xi * WK
                - vi = xi * WV
        - calcularea unor similaritati intre patch-ul curent (query-ul qi) si toate celelalte patch-uri (toate key-urile kj, j = 1,n): $[ qi * k1, qi * k2, \ldots, qi * k9]$
            - e o operatie invarianta la permutarea patch-urilor -> de aceea e nevoie de embedding-urile de pozitie (Care retin locatia intiala a patch-urilor)
            - e o operatie globala (care agrega informatii de la toate patch-urile) 
        - Normalizarea/scalarea similaritatii cu sqrt(D) - de ce? Pt a avea gradient mult mai stabili
        - Transformarea valorilor in ponderi (weights) prin aplicarea unui softmax -> scores
        - Maparea scorurilor peste valorile values asociate patch-urilor: Score1 *vi1, score 2 *vi2, …, score9 * vi9
        - agregarea rezultatelor: prin insumarea valorilor ponderate
- pentru un mecanism de atentie (cu mai multea head-uri = $noHEads$) am avea:
    - repetarea procesului anterior de #noHeads$ ori, dar in loc sa se foloseasca ca input toate cele De elemente ale unui embedding se vor folosi doar cate $De / noHeads$ elemente (se va face un "split" al embedding-ului in $noHeads$ parti) si se va aplica acelasi proces de mai sus pentru fiecare parte
    - stack-urirea rezultatelor obtinute de catre fiecare head <img  style="vertical-align:middle" src="images/ViT/mha.png" width="600"/>

    > in implementare avem clasa MyMSA in care se calculeaza query, key si value (liniile 44-52), se calculeaza produsul scalar intre query si key, se normalizeaza si se aplica softmax (linia 64-71). Maparea scorurilor peste valori si agregarea se face in linia 72.

## 5.3 Conexiuni reziduale

De ce e necesar?
- pentru a preveni "vanishing gradient problem" si pentru a imbunatati procesul de antrenare

Specificati:
- Input:
- Output:

Cum?
- adaugarea unor informatii (features la o anumita granularitate) la output-ul unui strat de procesare (de la o granularitate diferita), urmate de normalizare
    
    > in implementare, in clasa MyViTblock, linia 93 si 94


# Pas 6 - blocul de clasificare   <img  style="vertical-align:middle" src="images/ViT/classScores.png" width="600"/>

De ce e necesar?
- ca sa interpreteze embedding-urile produse de transformer (in special embedding-ul asociat token-ului [CLS] ) si sa le map-eze in label-uri

Specificatii:
- Input: un vector cu D elemente (embedding-ul token-ului [CLS] produs de transformer)
- Output: un vector cu C elemente (unde C este numarul de clase) de tip probabilitati cu suma 1 (confidentele in cele C clase)

Cum?
- folosirea unor layere fully connected peste embedding (un MLP)
    - activare ReLU sau GELU (Gaussian Error Linear Unit) - GELU nu are limitarile lui ReLU in ceea ce priveste ne-diferentiabilitatea in 0
- un ultim layer de tip softmax (problema de clasificare cu C clase)

    > in implementare, in clasa MyViT, linia 139 si 163